In [1]:
import json
from pprint import pprint
import tinydb as tdb

In [18]:
dbname = 'data/tinyDB.json'

In [20]:
!rm {dbname}


## OSM: Requete OverPass turbo:

```sql
[out:json][timeout:500];
{{geocodeArea:France}}->.boundaryarea;

(
 node(area.boundaryarea)["ref:mhs"];
 way(area.boundaryarea)["ref:mhs"];
 relation(area.boundaryarea)["ref:mhs"];
);

out tags;
```
-- 20Mo

Rq: j'ai essayé avec `out meta;`, le fichier fait ~200Mo, et avec `out body;` ~90Mo... c'est surement mieux de séparer par département...


In [21]:
# Charge le fichier JSON:
filename = 'data/osmTags_France.json' 

with open(filename) as f:
    dataOsm = json.load(f)
    
dataOsm = dataOsm['elements']
len(dataOsm)

33071

In [22]:
# Garde seulement les monuments avec le code commençant par PA...
dataOsm = [ el for el in dataOsm if el['tags']['ref:mhs'].startswith('PA') ]
len(dataOsm)

32246

In [23]:
def flatten( dico ):
    flatdico = dico['tags']
    flatdico['idOSM'] = dico['id']
    flatdico['typeOSM'] = dico['type']
    
    return flatdico

In [24]:
# Garde que les tags - and flatten the dict 
dataOsm = [ flatten( el ) for el in dataOsm ]

In [26]:
# Push to tinyDB
db = tdb.TinyDB(dbname)
db.purge_table('osm')
osm = db.table('osm')
ids = osm.insert_multiple(dataOsm)
len(ids)

32246

## Requete Wikidata
```sql
SELECT DISTINCT ?item ?itemDescription ?merimee ?article  WHERE {
  ?item wdt:P380 ?merimee .
  
  OPTIONAL { ?article schema:about ?item .
  ?article schema:isPartOf <https://fr.wikipedia.org/> . }
   
  FILTER(STRSTARTS(?merimee, "PA")).
  
  SERVICE wikibase:label {
     bd:serviceParam wikibase:language "fr" .
 }
}
```
-- 7,6 Mo

In [27]:
# Charge le fichier JSON:
filename = 'data/wikidata_France.json' 

with open(filename) as f:
    dataWiki = json.load(f)
    
len(dataWiki)

44912

In [28]:
dataWiki[0]

{'article': 'https://fr.wikipedia.org/wiki/%C3%89glise_Saint-%C3%89tienne-du-Mont_de_Paris',
 'item': 'http://www.wikidata.org/entity/Q583251',
 'itemDescription': 'édifice religieux de Paris, en France',
 'merimee': 'PA00088414'}

In [29]:
# Push to tinyDB
db = tdb.TinyDB(dbname)
db.purge_table('wikidata')
wikidata = db.table('wikidata')
ids = wikidata.insert_multiple(dataWiki)
len(ids)

44912

# Merimee

In [30]:
# -- Charge le fichier JSON: --
# src: https://www.data.gouv.fr/fr/datasets/immeubles-proteges-au-titre-des-monuments-historiques/

#filename = 'data/liste-des-immeubles-proteges-au-titre-des-monuments-historiques.json' 
filename = 'data/merimee-MH.json' 

with open(filename) as f:
    dataMerimee = json.load(f)
    
print( len(dataMerimee) )

# convert keys to lower-case:
dataMerimee = [ { k.lower():v for k, v in d.items() }  for d in  dataMerimee ]

45268


In [31]:
dataMerimee[1]

{'adrs': '',
 'affe': '',
 'autr': '',
 'com': 'Aix-en-Othe',
 'dpro': '1980/11/14 : inscrit MH',
 'dpt': '10',
 'etud': 'Recensement immeubles MH',
 'insee': '10003',
 'ppro': 'Eglise de la Nativité (cad. AB 179) : inscription par arrêté du 14 novembre 1980',
 'ref': 'PA00078015',
 'reg': 'Champagne-Ardenne',
 'scle': '16e siècle ; 17e siècle ; 18e siècle',
 'stat': 'Propriété de la commune',
 'tico': 'Eglise de la Nativité'}

In [32]:
# Push to tinyDB

db = tdb.TinyDB(dbname)
db.purge_table('merimee')
merimee = db.table('merimee')

ids = merimee.insert_multiple(dataMerimee)

len(ids)

45268